In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings("ignore")

# =========================
# CONFIG (à adapter)
# =========================
TEST_PATH = "/Users/alexandre/Desktop/X/Python for Data Science/Projet Final Churn/test.parquet"
EXAMPLE_PATH = "/Users/alexandre/Desktop/X/Python for Data Science/Projet Final Churn/example_submission.csv"

BUNDLE_FILE = "bundle_ensemble_v2.pkl"
OUTPUT_FILE = "submission_ensemble_v2.csv"

HORIZON_DAYS = 10
WINDOWS_DAYS = [3, 7, 14, 30]
KEY_PAGES = [
    "Thumbs Up", "Thumbs Down", "Roll Advert", "Error",
    "Upgrade", "Downgrade", "Add to Playlist", "Cancel"
]

In [2]:
print("⏳ Chargement du test...")
test_df = pd.read_parquet(TEST_PATH)

test_df["ts"] = pd.to_datetime(test_df["ts"], unit="ms", errors="coerce")
test_df["date"] = test_df["ts"].dt.date

T_test = test_df["ts"].max()
test_users = test_df["userId"].unique()

print("✅ test shape:", test_df.shape)
print("T_test:", T_test, "| nb users:", len(test_users))

⏳ Chargement du test...
✅ test shape: (4393179, 20)
T_test: 2018-11-20 00:00:00 | nb users: 2904


In [3]:
# =========================
# Helpers (copié de notebook 01 v2)
# =========================
def _is_song_event(d):
    if "page" in d.columns:
        return d["page"].eq("NextSong")
    if "length" in d.columns:
        return d["length"].notna()
    return pd.Series(False, index=d.index)

def _detect_os(user_agent):
    if pd.isna(user_agent): return "Unknown"
    ua = str(user_agent)
    if ("Mac" in ua) or ("iPhone" in ua) or ("iPad" in ua): return "Apple"
    if "Windows" in ua: return "Windows"
    if "Linux" in ua: return "Linux"
    return "Other"

def build_window_stats(obs, T0, window_days, suffix):
    start = T0 - pd.Timedelta(days=window_days)
    win = obs[obs["ts"] >= start].copy()
    if win.empty:
        return pd.DataFrame({"userId": obs["userId"].unique()})

    song_mask = _is_song_event(win)
    win_songs = win[song_mask].copy()

    agg = win.groupby("userId").agg(
        events_count=("ts","count"),
        sessions=("sessionId","nunique") if "sessionId" in win.columns else ("ts","count"),
        active_days=("date","nunique") if "date" in win.columns else ("ts","count"),
    ).reset_index()

    if "length" in win_songs.columns:
        lt = win_songs.groupby("userId")["length"].sum().reset_index(name="listen_time")
        agg = agg.merge(lt, on="userId", how="left")
    else:
        agg["listen_time"] = 0.0

    if "artist" in win_songs.columns:
        ua = win_songs.groupby("userId")["artist"].nunique().reset_index(name="uniq_artists")
        agg = agg.merge(ua, on="userId", how="left")
    else:
        agg["uniq_artists"] = 0

    if "song" in win_songs.columns:
        us = win_songs.groupby("userId")["song"].nunique().reset_index(name="uniq_songs")
        agg = agg.merge(us, on="userId", how="left")
    else:
        agg["uniq_songs"] = 0

    agg["listen_per_active_day"] = agg["listen_time"] / (agg["active_days"] + 1e-6)
    agg["songs_per_session"] = agg["events_count"] / (agg["sessions"] + 1e-6)

    rename = {c: f"{c}_{suffix}" for c in agg.columns if c != "userId"}
    agg = agg.rename(columns=rename).fillna(0)
    return agg

def add_page_counts(obs, users):
    if "page" not in obs.columns:
        return pd.DataFrame({"userId": users})
    page_counts = pd.pivot_table(
        obs, index="userId", columns="page", values="ts", aggfunc="count", fill_value=0
    ).reset_index()
    keep = ["userId"] + [p for p in KEY_PAGES if p in page_counts.columns]
    out = page_counts[keep].copy()

    if ("Thumbs Up" in out.columns) and ("Thumbs Down" in out.columns):
        out["satisfaction_ratio"] = out["Thumbs Up"] / (out["Thumbs Down"] + 1)
    if "Roll Advert" in out.columns:
        out["ad_events"] = out["Roll Advert"]
    if "Error" in out.columns:
        out["error_events"] = out["Error"]

    return out

def add_recency_features(obs, T0, users):
    out = pd.DataFrame({"userId": users})
    for p in KEY_PAGES:
        colname = f"recency_{p.replace(' ','_').lower()}"
        if "page" not in obs.columns:
            out[colname] = 999
            continue
        last = obs[obs["page"] == p].groupby("userId")["ts"].max().reset_index(name="last")
        last[colname] = (T0 - last["last"]).dt.total_seconds() / 86400.0
        last = last.drop(columns=["last"])
        out = out.merge(last, on="userId", how="left")
        out[colname] = out[colname].fillna(999)
    return out

def add_session_stats(obs):
    if "sessionId" not in obs.columns:
        return pd.DataFrame({"userId": obs["userId"].unique()})
    g = obs.groupby(["userId","sessionId"]).agg(
        sess_events=("ts","count"),
        sess_start=("ts","min"),
        sess_end=("ts","max"),
        sess_listen=("length","sum") if "length" in obs.columns else ("ts","count")
    ).reset_index()
    g["sess_duration_min"] = (g["sess_end"] - g["sess_start"]).dt.total_seconds() / 60.0

    agg = g.groupby("userId").agg(
        sess_events_mean=("sess_events","mean"),
        sess_events_std=("sess_events","std"),
        sess_duration_mean=("sess_duration_min","mean"),
        sess_duration_std=("sess_duration_min","std"),
        sess_listen_mean=("sess_listen","mean"),
        sess_listen_std=("sess_listen","std"),
        sess_listen_max=("sess_listen","max"),
    ).reset_index()
    return agg.fillna(0)

def add_level_features(obs, users):
    if "level" not in obs.columns:
        return pd.DataFrame({"userId": users})
    last_level = obs.sort_values("ts").groupby("userId")["level"].last().reset_index(name="level_last")
    changes = obs.sort_values("ts").groupby("userId")["level"].apply(lambda s: (s != s.shift(1)).sum()).reset_index(name="level_changes")
    out = pd.DataFrame({"userId": users}).merge(last_level, on="userId", how="left").merge(changes, on="userId", how="left")
    return out

def add_demo_features(obs, users):
    out = pd.DataFrame({"userId": users})
    for col in ["gender"]:
        if col in obs.columns:
            last = obs.sort_values("ts").groupby("userId")[col].last().reset_index(name=f"{col}_last")
            out = out.merge(last, on="userId", how="left")
    return out

In [4]:
# =========================
# Compute test features (mêmes features que train)
# =========================
obs = test_df[test_df["ts"] <= T_test].copy()
users = test_users

song_mask = _is_song_event(obs)
obs_songs = obs[song_mask].copy()

global_feats = obs.groupby("userId").agg(
    n_active_days=("date","nunique") if "date" in obs.columns else ("ts","count"),
    n_sessions=("sessionId","nunique") if "sessionId" in obs.columns else ("ts","count"),
    n_events=("ts","count"),
).reset_index()

if "length" in obs_songs.columns:
    lt = obs_songs.groupby("userId")["length"].sum().reset_index(name="total_listening_time")
    global_feats = global_feats.merge(lt, on="userId", how="left")
else:
    global_feats["total_listening_time"] = 0.0

if "artist" in obs_songs.columns:
    ua = obs_songs.groupby("userId")["artist"].nunique().reset_index(name="uniq_artists_global")
    global_feats = global_feats.merge(ua, on="userId", how="left")
else:
    global_feats["uniq_artists_global"] = 0

if "song" in obs_songs.columns:
    us = obs_songs.groupby("userId")["song"].nunique().reset_index(name="uniq_songs_global")
    global_feats = global_feats.merge(us, on="userId", how="left")
else:
    global_feats["uniq_songs_global"] = 0

# recency/account age
last_ts = obs.groupby("userId")["ts"].max().reset_index(name="last_ts")
global_feats = global_feats.merge(last_ts, on="userId", how="left")

if "registration" in obs.columns:
    reg = obs.groupby("userId")["registration"].min().reset_index(name="registration_ts")
    reg["registration_ts"] = pd.to_datetime(reg["registration_ts"], unit="ms", errors="coerce")
    global_feats = global_feats.merge(reg, on="userId", how="left")
    global_feats["account_age_days"] = (T_test - global_feats["registration_ts"]).dt.total_seconds() / 86400.0
else:
    global_feats["account_age_days"] = np.nan

global_feats["recency_days"] = (T_test - global_feats["last_ts"]).dt.total_seconds() / 86400.0
global_feats["avg_daily_listen"] = global_feats["total_listening_time"] / (global_feats["account_age_days"].fillna(0) + 1)
global_feats["sessions_per_day"] = global_feats["n_sessions"] / (global_feats["n_active_days"] + 1e-6)
global_feats["events_per_session"] = global_feats["n_events"] / (global_feats["n_sessions"] + 1e-6)
global_feats["uniq_songs_per_day"] = global_feats["uniq_songs_global"] / (global_feats["n_active_days"] + 1e-6)

global_feats = global_feats.drop(columns=[c for c in ["last_ts","registration_ts"] if c in global_feats.columns])

# windows
windows_df = pd.DataFrame({"userId": users})
for w in WINDOWS_DAYS:
    windows_df = windows_df.merge(build_window_stats(obs, T_test, w, f"{w}d"), on="userId", how="left")
windows_df = windows_df.fillna(0)

if "listen_time_7d" in windows_df.columns and "listen_time_14d" in windows_df.columns:
    windows_df["ratio_listen_7d_14d"] = windows_df["listen_time_7d"] / (windows_df["listen_time_14d"] + 1)
if "listen_time_3d" in windows_df.columns and "listen_time_14d" in windows_df.columns:
    windows_df["ratio_listen_3d_14d"] = windows_df["listen_time_3d"] / (windows_df["listen_time_14d"] + 1)

# behavior + recency pages
behavior_df = add_page_counts(obs, users)
recency_df  = add_recency_features(obs, T_test, users)

# trend
recent = obs[obs["ts"] >= (T_test - pd.Timedelta(days=14))].copy()
if not recent.empty and "length" in recent.columns:
    recent_songs = recent[_is_song_event(recent)]
    recent_listen = recent_songs.groupby("userId")["length"].sum().reset_index(name="listen_time_recent_14d")
else:
    recent_listen = pd.DataFrame({"userId": users, "listen_time_recent_14d": 0.0})

trends = pd.DataFrame({"userId": users}).merge(recent_listen, on="userId", how="left").fillna(0)
trends = trends.merge(global_feats[["userId","avg_daily_listen"]], on="userId", how="left").fillna(0)
trends["avg_daily_listen_recent_14d"] = trends["listen_time_recent_14d"] / 14.0
trends["trend_listening"] = trends["avg_daily_listen_recent_14d"] / (trends["avg_daily_listen"] + 1e-6)
trends = trends[["userId","trend_listening"]]

# session stats
session_stats = add_session_stats(obs)

# tech OS
if "userAgent" in obs.columns:
    last_agent = obs.sort_values("ts").groupby("userId")["userAgent"].last().reset_index()
    last_agent["os_type"] = last_agent["userAgent"].apply(_detect_os)
    tech = pd.get_dummies(last_agent[["userId","os_type"]], columns=["os_type"], prefix="os")
else:
    tech = pd.DataFrame({"userId": users})

# level + demo
level_df = add_level_features(obs, users)
demo_df  = add_demo_features(obs, users)

X_test = (pd.DataFrame({"userId": users})
    .merge(global_feats, on="userId", how="left")
    .merge(windows_df, on="userId", how="left")
    .merge(behavior_df, on="userId", how="left")
    .merge(recency_df, on="userId", how="left")
    .merge(trends, on="userId", how="left")
    .merge(session_stats, on="userId", how="left")
    .merge(tech, on="userId", how="left")
    .merge(level_df, on="userId", how="left")
    .merge(demo_df, on="userId", how="left")
).fillna(0)

userId_col = X_test["userId"].copy()
X_test = X_test.drop(columns=["userId"])

# one-hot categoricals
cat_cols = X_test.select_dtypes(include=["object","category"]).columns.tolist()
if cat_cols:
    X_test = pd.get_dummies(X_test, columns=cat_cols, dummy_na=True)

for c in X_test.select_dtypes(include=["bool"]).columns:
    X_test[c] = X_test[c].astype(int)

X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

print("✅ X_test built:", X_test.shape)

✅ X_test built: (2904, 84)


In [ ]:
import json
import numpy as np
import xgboost as xgb
import pandas as pd

def predict_best(booster, dmat):
    if hasattr(booster, "best_ntree_limit") and booster.best_ntree_limit:
        return booster.predict(dmat, ntree_limit=booster.best_ntree_limit)
    if hasattr(booster, "best_iteration") and booster.best_iteration is not None:
        try:
            return booster.predict(dmat, iteration_range=(0, booster.best_iteration + 1))
        except TypeError:
            return booster.predict(dmat, ntree_limit=booster.best_iteration + 1)
    return booster.predict(dmat)

# ---- load bundle ----
with open("xgb_ensemble_v3/bundle.json", "r") as f:
    bundle = json.load(f)

cols = bundle["features"]
th = float(bundle["threshold"])
w_full = float(bundle["w_full"])
paths_full = bundle["model_paths_full"]
paths_recent = bundle["model_paths_recent"]

# ---- align columns ----
X_test = X_test.reindex(columns=cols, fill_value=0)

dtest = xgb.DMatrix(X_test.values, missing=np.nan)

import json
import numpy as np
import xgboost as xgb
import pandas as pd
import os

BUNDLE_PATH = "xgb_ensemble_v4/bundle.json"
OUT_SUB_DIR = "subs_sweep_v4"
os.makedirs(OUT_SUB_DIR, exist_ok=True)

def load_booster(path):
    b = xgb.Booster()
    b.load_model(path)
    return b

# ---- load bundle ----
with open(BUNDLE_PATH, "r") as f:
    bundle = json.load(f)

cols = bundle["features"]
paths = bundle["model_paths"]

# ---- align ----
X_test = X_test.reindex(columns=cols, fill_value=0)
dtest = xgb.DMatrix(X_test.values, missing=np.nan)

# ---- predict mean proba across all models ----
models = [load_booster(p) for p in paths]
all_p = np.vstack([m.predict(dtest) for m in models])  # (n_models, n_samples)

# rank par modèle -> [0,1], puis moyenne des ranks
ranks = np.argsort(np.argsort(all_p, axis=1), axis=1).astype(float)
ranks = ranks / (ranks.shape[1] - 1 + 1e-12)

probs = ranks.mean(axis=0)

print("probs(rank-avg) min/mean/median/max:",
      float(np.min(probs)), float(np.mean(probs)), float(np.median(probs)), float(np.max(probs)))


# ---- helper: save aligned to example_submission ----
example = pd.read_csv(EXAMPLE_PATH)
example["id"] = example["id"].astype(str)

base_df = pd.DataFrame({"id": userId_col.astype(str), "proba": probs})

def save_preds(preds, name):
    sub = pd.DataFrame({"id": base_df["id"], "target": preds.astype(int)})
    final = example[["id"]].merge(sub, on="id", how="left")
    final["target"] = final["target"].fillna(0).astype(int)
    out = os.path.join(OUT_SUB_DIR, name)
    final.to_csv(out, index=False)
    print("✅", name, "| churn rate =", float(final["target"].mean()))
    return out

# ---- A) threshold sweep ----
thresholds = [0.03, 0.04, 0.05, 0.06, 0.08, 0.10, 0.12, 0.15, 0.20, 0.25, 0.30, 0.35, 0.38, 0.40, 0.45, 0.50, 0.60]
for t in thresholds:
    preds = (base_df["proba"].values >= t).astype(int)
    save_preds(preds, f"sub_thr_{str(t).replace('.','p')}.csv")

# ---- B) top-K sweep (prend les K% plus hauts scores) ----
for pct in [0.05, 0.08, 0.10, 0.12, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40]:
    k = int(np.ceil(len(base_df) * pct))
    idx = np.argsort(-base_df["proba"].values)[:k]
    preds = np.zeros(len(base_df), dtype=int)
    preds[idx] = 1
    save_preds(preds, f"sub_top_{int(pct*100)}pct.csv")

print("\n📦 Submissions générées dans:", OUT_SUB_DIR)

probs min/mean/median/max: 0.0034335912205278873 0.08661282062530518 0.06182315945625305 0.6620886325836182
✅ sub_thr_0p03.csv | churn rate = 0.78133608815427
✅ sub_thr_0p04.csv | churn rate = 0.6821625344352618
✅ sub_thr_0p05.csv | churn rate = 0.5884986225895317
✅ sub_thr_0p06.csv | churn rate = 0.5137741046831956
✅ sub_thr_0p08.csv | churn rate = 0.37706611570247933
✅ sub_thr_0p1.csv | churn rate = 0.2830578512396694
✅ sub_thr_0p12.csv | churn rate = 0.22107438016528927
✅ sub_thr_0p15.csv | churn rate = 0.1415289256198347
✅ sub_thr_0p2.csv | churn rate = 0.07851239669421488
✅ sub_thr_0p25.csv | churn rate = 0.0506198347107438
✅ sub_thr_0p3.csv | churn rate = 0.03581267217630854
✅ sub_thr_0p35.csv | churn rate = 0.02203856749311295
✅ sub_thr_0p38.csv | churn rate = 0.01721763085399449
✅ sub_thr_0p4.csv | churn rate = 0.013085399449035813
✅ sub_thr_0p45.csv | churn rate = 0.007231404958677686
✅ sub_thr_0p5.csv | churn rate = 0.0030991735537190084
✅ sub_thr_0p6.csv | churn rate = 0.001

In [6]:
import numpy as np
import pandas as pd
import os

OUT_SUB_DIR = "subs_refine_topk"
os.makedirs(OUT_SUB_DIR, exist_ok=True)

# base_df doit exister: colonnes ["id","proba"] (id=str, proba=float)
# example = example_submission chargé avec colonne "id" (str)

def save_preds(preds, name):
    sub = pd.DataFrame({"id": base_df["id"], "target": preds.astype(int)})
    final = example[["id"]].merge(sub, on="id", how="left")
    final["target"] = final["target"].fillna(0).astype(int)
    out = os.path.join(OUT_SUB_DIR, name)
    final.to_csv(out, index=False)
    print("✅", name, "| churn rate =", float(final["target"].mean()))
    return out

# 🔥 sweep fin
for pct in [0.37, 0.38, 0.39, 0.395, 0.40, 0.405, 0.41, 0.42, 0.43, 0.44, 0.45]:
    k = int(np.ceil(len(base_df) * pct))
    idx = np.argsort(-base_df["proba"].values)[:k]
    preds = np.zeros(len(base_df), dtype=int)
    preds[idx] = 1
    save_preds(preds, f"sub_top_{str(pct).replace('.','p')}pct.csv")

print("📦 Fichiers dans:", OUT_SUB_DIR)

✅ sub_top_0p37pct.csv | churn rate = 0.3701790633608815
✅ sub_top_0p38pct.csv | churn rate = 0.38016528925619836
✅ sub_top_0p39pct.csv | churn rate = 0.39015151515151514
✅ sub_top_0p395pct.csv | churn rate = 0.3953168044077135
✅ sub_top_0p4pct.csv | churn rate = 0.400137741046832
✅ sub_top_0p405pct.csv | churn rate = 0.4053030303030303
✅ sub_top_0p41pct.csv | churn rate = 0.41012396694214875
✅ sub_top_0p42pct.csv | churn rate = 0.4201101928374656
✅ sub_top_0p43pct.csv | churn rate = 0.43009641873278237
✅ sub_top_0p44pct.csv | churn rate = 0.44008264462809915
✅ sub_top_0p45pct.csv | churn rate = 0.450068870523416
📦 Fichiers dans: subs_refine_topk


In [9]:
import json
import numpy as np
import pandas as pd
import xgboost as xgb

# ---- chemins ----
BUNDLE_PATH = "xgb_ensemble_v4/bundle.json"   # bundle créé par le notebook 02
# EXAMPLE_PATH doit déjà exister dans ton notebook (example_submission.csv)

# ---- 1) charger example_submission ----
example = pd.read_csv(EXAMPLE_PATH)
example["id"] = example["id"].astype(str)

# ---- 2) charger bundle et aligner colonnes ----
with open(BUNDLE_PATH, "r") as f:
    bundle = json.load(f)

cols = bundle["features"]
paths = bundle["model_paths"]

X_test = X_test.reindex(columns=cols, fill_value=0)
dtest = xgb.DMatrix(X_test.values, missing=np.nan)

# ---- 3) charger les modèles ----
def load_booster(path):
    b = xgb.Booster()
    b.load_model(path)
    return b

models = [load_booster(p) for p in paths]

# ---- 4) rank-average (au lieu de mean proba) ----
all_p = np.vstack([m.predict(dtest) for m in models])  # (n_models, n_samples)

ranks = np.argsort(np.argsort(all_p, axis=1), axis=1).astype(float)
ranks = ranks / (ranks.shape[1] - 1 + 1e-12)

probs = ranks.mean(axis=0)  # score pour le tri Top-K

print("score(rank-avg) min/mean/median/max:",
      float(np.min(probs)), float(np.mean(probs)), float(np.median(probs)), float(np.max(probs)))

# ---- 5) base_df (id + score) ----
base_df = pd.DataFrame({
    "id": userId_col.astype(str),
    "proba": probs
})

print("✅ base_df ready:", base_df.shape)


score(rank-avg) min/mean/median/max: 0.0004133654839820874 0.49999999999999983 0.5042197726489837 0.9989665862900445
✅ base_df ready: (2904, 2)


In [8]:
import numpy as np
import pandas as pd
import os

OUT_SUB_DIR = "subs_topk_refine_only"
os.makedirs(OUT_SUB_DIR, exist_ok=True)

def save_topk(pct):
    k = int(np.ceil(len(base_df) * pct))
    idx = np.argsort(-base_df["proba"].values)[:k]
    preds = np.zeros(len(base_df), dtype=int)
    preds[idx] = 1

    sub = pd.DataFrame({"id": base_df["id"], "target": preds.astype(int)})
    final = example[["id"]].merge(sub, on="id", how="left")
    final["target"] = final["target"].fillna(0).astype(int)

    name = f"sub_top_{str(pct).replace('.','p')}pct.csv"
    out = os.path.join(OUT_SUB_DIR, name)
    final.to_csv(out, index=False)

    print("✅", name, "| churn rate =", float(final["target"].mean()))
    return out

pcts = [0.37, 0.38, 0.39, 0.395, 0.40, 0.405, 0.41, 0.42, 0.43, 0.44, 0.45]
for pct in pcts:
    save_topk(pct)

print("\n📦 Fichiers générés dans:", OUT_SUB_DIR)
print("👉 À soumettre en priorité : 0.39, 0.395, 0.40, 0.405, 0.41, 0.42")

✅ sub_top_0p37pct.csv | churn rate = 0.3701790633608815
✅ sub_top_0p38pct.csv | churn rate = 0.38016528925619836
✅ sub_top_0p39pct.csv | churn rate = 0.39015151515151514
✅ sub_top_0p395pct.csv | churn rate = 0.3953168044077135
✅ sub_top_0p4pct.csv | churn rate = 0.400137741046832
✅ sub_top_0p405pct.csv | churn rate = 0.4053030303030303
✅ sub_top_0p41pct.csv | churn rate = 0.41012396694214875
✅ sub_top_0p42pct.csv | churn rate = 0.4201101928374656
✅ sub_top_0p43pct.csv | churn rate = 0.43009641873278237
✅ sub_top_0p44pct.csv | churn rate = 0.44008264462809915
✅ sub_top_0p45pct.csv | churn rate = 0.450068870523416

📦 Fichiers générés dans: subs_topk_refine_only
👉 À soumettre en priorité : 0.39, 0.395, 0.40, 0.405, 0.41, 0.42


In [10]:
import numpy as np
import pandas as pd
import os

OUT_SUB_DIR = "subs_topk_micro_039"
os.makedirs(OUT_SUB_DIR, exist_ok=True)

def save_topk(pct):
    k = int(np.ceil(len(base_df) * pct))
    idx = np.argsort(-base_df["proba"].values)[:k]
    preds = np.zeros(len(base_df), dtype=int)
    preds[idx] = 1

    sub = pd.DataFrame({"id": base_df["id"], "target": preds.astype(int)})
    final = example[["id"]].merge(sub, on="id", how="left")
    final["target"] = final["target"].fillna(0).astype(int)

    name = f"sub_top_{str(pct).replace('.','p')}pct.csv"
    out = os.path.join(OUT_SUB_DIR, name)
    final.to_csv(out, index=False)

    print("✅", name, "| churn rate =", float(final["target"].mean()))
    return out

# micro sweep autour de 0.39
pcts = [0.387, 0.388, 0.389, 0.390, 0.391, 0.392, 0.393]
for pct in pcts:
    save_topk(pct)

print("\n📦 Fichiers générés dans:", OUT_SUB_DIR)
print("👉 À soumettre en priorité : 0.389, 0.390, 0.391")


✅ sub_top_0p387pct.csv | churn rate = 0.38705234159779617
✅ sub_top_0p388pct.csv | churn rate = 0.3880853994490358
✅ sub_top_0p389pct.csv | churn rate = 0.3891184573002755
✅ sub_top_0p39pct.csv | churn rate = 0.39015151515151514
✅ sub_top_0p391pct.csv | churn rate = 0.39118457300275483
✅ sub_top_0p392pct.csv | churn rate = 0.39221763085399447
✅ sub_top_0p393pct.csv | churn rate = 0.39325068870523416

📦 Fichiers générés dans: subs_topk_micro_039
👉 À soumettre en priorité : 0.389, 0.390, 0.391


In [11]:
import json
import numpy as np
import pandas as pd
import xgboost as xgb
import os

# ====== CONFIG FINALS ======
BUNDLE_PATH = "xgb_ensemble_v4/bundle.json"     # <-- ton ensemble final
FINAL1_TOPK = 0.39                              # winner public
FINAL2_TOPK = 0.40                              # hedge private
OUT1 = "FINAL_1_rankavg_top_0p39.csv"
OUT2 = "FINAL_2_meanproba_top_0p40.csv"
# ===========================

# load example (pour l'ordre des ids)
example = pd.read_csv(EXAMPLE_PATH)
example["id"] = example["id"].astype(str)

# load bundle
with open(BUNDLE_PATH, "r") as f:
    bundle = json.load(f)

cols = bundle["features"]
paths = bundle["model_paths"]

# align + DMatrix
X_test_aligned = X_test.reindex(columns=cols, fill_value=0)
dtest = xgb.DMatrix(X_test_aligned.values, missing=np.nan)

# load models
def load_booster(path):
    b = xgb.Booster()
    b.load_model(path)
    return b

models = [load_booster(p) for p in paths]
all_p = np.vstack([m.predict(dtest) for m in models])  # (n_models, n_samples)

# ---------- SCORE 1: rank-average ----------
ranks = np.argsort(np.argsort(all_p, axis=1), axis=1).astype(float)
ranks = ranks / (ranks.shape[1] - 1 + 1e-12)
score_rankavg = ranks.mean(axis=0)

# ---------- SCORE 2: mean-proba ----------
score_meanproba = all_p.mean(axis=0)

base_rank = pd.DataFrame({"id": userId_col.astype(str), "score": score_rankavg})
base_mean = pd.DataFrame({"id": userId_col.astype(str), "score": score_meanproba})

def save_topk(base_df, pct, out_name):
    k = int(np.ceil(len(base_df) * pct))
    idx = np.argsort(-base_df["score"].values)[:k]
    preds = np.zeros(len(base_df), dtype=int)
    preds[idx] = 1

    sub = pd.DataFrame({"id": base_df["id"], "target": preds})
    final = example[["id"]].merge(sub, on="id", how="left")
    final["target"] = final["target"].fillna(0).astype(int)
    final.to_csv(out_name, index=False)

    print("✅ saved", out_name, "| churn rate =", float(final["target"].mean()))

save_topk(base_rank, FINAL1_TOPK, OUT1)
save_topk(base_mean, FINAL2_TOPK, OUT2)


✅ saved FINAL_1_rankavg_top_0p39.csv | churn rate = 0.39015151515151514
✅ saved FINAL_2_meanproba_top_0p40.csv | churn rate = 0.400137741046832


In [12]:
FINAL2_TOPK = 0.39
OUT2 = "FINAL_2_meanproba_top_0p39.csv"
save_topk(base_mean, FINAL2_TOPK, OUT2)


✅ saved FINAL_2_meanproba_top_0p39.csv | churn rate = 0.39015151515151514
